## Globals

In [1]:
from __future__ import print_function

import sys
import os
import re
import glob
import subprocess
import shutil
import json

#root_dir = '/home/preclineu/yartoe'
root_dir = '/project_cephfs/3022017.06/OASIS3/'
cluster_log_dir = os.path.join(root_dir,'logs')
#aroma_path = '/home/preclineu/andmar/sfw/ICA-AROMA/ICA_AROMA.py'  

cmd_qsub_base = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '102400',
                 '-memory', '32gb',
                 '-logfiledir', cluster_log_dir
                ]

#abcd_dict = os.path.join(root_dir,'downloads','abcd_modalities.json')

# read the dictionary
#with open(abcd_dict,'r') as f:
   #modalities = json.load(f) 
    
# get a list of subjects to process
sub_dirs = sorted(glob.glob(os.path.join(root_dir,'subjects','*_MR*')))

#sub_dirs = sub_dirs[0:15]
#print('\n'.join(sub_dirs))

#print('Dictionary read from:', abcd_dict)
print('Found', len(sub_dirs), 'subjects to process')
sub_dirs

Found 2162 subjects to process


['/project_cephfs/3022017.06/OASIS3/subjects/OAS30001_MR_d0129',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30001_MR_d0757',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30001_MR_d2430',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30001_MR_d3132',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30002_MR_d0371',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30002_MR_d0653',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30002_MR_d2340',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30002_MR_d2345',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30003_MR_d0558',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30003_MR_d1631',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30003_MR_d2669',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30003_MR_d2682',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30003_MR_d3320',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30003_MR_d3731',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30004_MR_d1101',
 '/project_cephfs/3022017

## run fsl_anat

In [17]:
verbose = False

sub_batch = sub_dirs[:10]#[10000:]

submitted = 0
already_processed = 0
for s in sub_batch:
    subid = os.path.splitext(os.path.basename(s))[0]
    if verbose:
        print('processing subject',subid)

    job_name = 'ABCD_fsl_anat_' + subid
    
    sessions = glob.glob(os.path.join(s, 'ses-*'))
    for sess in sessions:
        sdir = os.path.join(sess, 'anat')
        if not os.path.exists(sdir):
            continue
        
        # get all subdirectories 
        t1_all = [os.path.join(sdir, o) for o in os.listdir(sdir) if os.path.isdir(os.path.join(sdir, o))]

        for t1_dir in t1_all:
            if os.path.exists(os.path.join(t1_dir,'T1.anat','T1_biascorr_brain.nii.gz')):
                if verbose: 
                    print(' >', subid, ' anatomical data already processed. Doing nothing')
                already_processed += 1
                continue
    
            cmd_fsl_anat = ['fsl_anat -i ', os.path.join(t1_dir,'T1.nii.gz')]
            cmd_str = '"%s"' % str(' '.join(cmd_fsl_anat)) 
            cmd_qsub = cmd_qsub_base + ['-name', job_name,'-command', cmd_str]

            #subprocess.Popen(' '.join(cmd_qsub), shell=True)
            
            submitted += 1
    
print('number of jobs submitted =', submitted)
print('number of T1s already processed =', already_processed)

number of jobs submitted = 4
number of T1s already processed = 24


## Select subjects (e.g. only those having 4 year follow-up)

In [3]:
target_ses = '_MR*'
#target_ses = '_MRI2'
#target_ses = '_MRI3'

sub_list = []
sub_dirs_target = []
sub_list_short = []

#for s in sub_dirs:
#    subid = os.path.splitext(os.path.basename(s))[0] 
#    ses_dirs = glob.glob(os.path.join(s))
    #print(ses_dirs)
#    ses_dirs_short = os.path.join(('/project_' + s.split("_")[1]))
#    sub_list_short.append(ses_dirs_short)
    
#    sessions = list(map(os.path.basename, ses_dirs))
#    if target_ses in sessions:
#        sub_list.append(subid)
#        sub_dirs_target.append(s)

#print(len(sub_list), 'subjects found')


for s in sub_dirs:
    subid = os.path.splitext(os.path.basename(s))[0]
    subid_path = os.path.dirname(s) +"/" + subid.split("_")[0] + "_" + subid.split("_")[1]
    subid_short = subid.split("_")[0] + "_" + subid.split("_")[1]
    sub_list_short.append(subid_short)
    sub_list.append(subid_path)

sub_list
sub_list_short

['OAS30001_MR',
 'OAS30001_MR',
 'OAS30001_MR',
 'OAS30001_MR',
 'OAS30002_MR',
 'OAS30002_MR',
 'OAS30002_MR',
 'OAS30002_MR',
 'OAS30003_MR',
 'OAS30003_MR',
 'OAS30003_MR',
 'OAS30003_MR',
 'OAS30003_MR',
 'OAS30003_MR',
 'OAS30004_MR',
 'OAS30004_MR',
 'OAS30004_MR',
 'OAS30004_MR',
 'OAS30005_MR',
 'OAS30005_MR',
 'OAS30005_MR',
 'OAS30005_MR',
 'OAS30005_MR',
 'OAS30005_MR',
 'OAS30006_MR',
 'OAS30006_MR',
 'OAS30006_MR',
 'OAS30006_MR',
 'OAS30006_MR',
 'OAS30007_MR',
 'OAS30007_MR',
 'OAS30007_MR',
 'OAS30007_MR',
 'OAS30007_MR',
 'OAS30008_MR',
 'OAS30008_MR',
 'OAS30008_MR',
 'OAS30008_MR',
 'OAS30009_MR',
 'OAS30009_MR',
 'OAS30009_MR',
 'OAS30010_MR',
 'OAS30011_MR',
 'OAS30011_MR',
 'OAS30012_MR',
 'OAS30012_MR',
 'OAS30013_MR',
 'OAS30014_MR',
 'OAS30014_MR',
 'OAS30015_MR',
 'OAS30015_MR',
 'OAS30015_MR',
 'OAS30015_MR',
 'OAS30016_MR',
 'OAS30017_MR',
 'OAS30018_MR',
 'OAS30018_MR',
 'OAS30019_MR',
 'OAS30020_MR',
 'OAS30020_MR',
 'OAS30021_MR',
 'OAS30022_MR',
 'OAS300

In [4]:
# change the subject list for OASIS
#sub_list[:10]
#os.path.join('/project', s.split("_")[1])
single_subs = set(sub_list_short)
single_subs = list(single_subs)
single_subs

sub_batch = set(sub_list)
sub_batch = list(sub_batch)
sub_batch


['/project_cephfs/3022017.06/OASIS3/subjects/OAS30923_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30350_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30362_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30353_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30864_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30599_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30292_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30235_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30267_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30306_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30030_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS31118_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS31105_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30385_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30010_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30441_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30174_MR

## Run Freesurfer (cross-sectional)

In [42]:
num_processed = 0
num_failed = 0
num_submitted = 0
sub_no_t1 = []

verbose = True

default_t1_type = 't1.nii.gz'
log_prefix = 'OASIS3_fs_rerun'

#sub_batch = sub_dirs[100:]
sub_batch = sub_dirs

#fs_env = 'export FREESURFER_HOME=/opt/freesurfer/7.3.2'
fs_env = 'module load freesurfer/7.3.2'

cmd_qsub_fs = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '202400',
                 '-memory', '5gb',
                 '-logfiledir', cluster_log_dir,
                 '-queue verylong'
                ]

print('Processing', len(sub_batch), 'subjects ...')
for s in sub_batch:
    subid = os.path.splitext(os.path.basename(s))[0] 
    full_subid = subid
    #sessions = glob.glob(os.path.join(s, 'ses-*'))
    #for sess in sessions:
        #sdir = os.path.join(sess, 'anat')
    sdir = os.path.join(s,'anat2')
        #sesid = os.path.basename(sess)
        #full_subid = '_'.join((subid, sesid))
        
    #if not os.path.exists(sdir):
        #continue
        
        # get all subdirectories 
    #t1_all = [os.path.join(sdir, o) for o in os.listdir(sdir) if os.path.isdir(os.path.join(sdir, o))]
    #t1_all = sdir
     
    #if len(t1_all) > 1:
        #if verbose:
            #print(' > Multiple t1s found for', full_subid, '(',len(t1_all), ') using default')
            
            # sub-select only ABCD-T1-NORM
        #t1_all = [k for k in t1_all if default_t1_type in k]
        #if len(t1_all) > 1:
            #if verbose:
                #print(' > Failed.')
            #continue;                        
        
    #for t1_dir in sdir:
    t1_file = os.path.join(sdir,'t1.nii.gz')            
        #out_dir = os.path.join(root_dir, 'freesurfer', full_subid)
    out_dir = os.path.join(root_dir, 'freesurfer' , full_subid)        
    if os.path.exists(out_dir):
        if os.path.exists(os.path.join(out_dir, 'stats', 'rh.w-g.pct.stats')):
                    # analysis is complete
            if verbose:
                print(' > Freesurfer already run for', subid,'doing nothing')
            num_processed += 1
            continue
        else:
                    # analysis did not complete
            num_failed +=1
            if verbose:
                print(' > Freesurfer subject',full_subid,'found but failed. Continuing analysis')
            cmd_fs = ['recon-all -s', full_subid, '-all','-qcache']
    else:
                # no analysis has been run
        if not os.path.exists(t1_file):
            if verbose: 
                print(' > No T1 found for sub', full_subid, '. Doing nothing')
            sub_no_t1.append(subid)
            continue
        
        cmd_fs = ['recon-all -i ', t1_file, '-s', full_subid, '-all','-qcache']
            
            #print(' '.join(cmd_fs))

    cmd_env = [fs_env + ' ; ',
                    'export SUBJECTS_DIR=' + os.path.join(root_dir,'freesurfer ; ')]
    cmd_str = '"%s"' % str(' '.join(cmd_env + cmd_fs)) 
    cmd_qsub = cmd_qsub_fs + ['-name', log_prefix + str(full_subid),'-command', cmd_str]
    
    num_submitted += 1
    #print(' '.join(cmd_qsub))
    subprocess.Popen(' '.join(cmd_qsub), shell=True)

print('jobs submitted =', num_submitted)       
print('number of subjects already processed =',num_processed)
print('number failed =',num_failed)
print('number without t1 =',len(sub_no_t1))

Processing 2162 subjects ...
 > Freesurfer already run for OAS30001_MR_d0129 doing nothing
 > Freesurfer already run for OAS30001_MR_d0757 doing nothing
 > Freesurfer already run for OAS30001_MR_d2430 doing nothing
 > Freesurfer already run for OAS30001_MR_d3132 doing nothing
 > No T1 found for sub OAS30002_MR_d0371 . Doing nothing
 > Freesurfer already run for OAS30002_MR_d0653 doing nothing
 > Freesurfer already run for OAS30002_MR_d2340 doing nothing
 > Freesurfer already run for OAS30002_MR_d2345 doing nothing
 > Freesurfer already run for OAS30003_MR_d0558 doing nothing
 > Freesurfer already run for OAS30003_MR_d1631 doing nothing
 > Freesurfer already run for OAS30003_MR_d2669 doing nothing
 > Freesurfer already run for OAS30003_MR_d2682 doing nothing
 > Freesurfer already run for OAS30003_MR_d3320 doing nothing
 > Freesurfer already run for OAS30003_MR_d3731 doing nothing
 > Freesurfer already run for OAS30004_MR_d1101 doing nothing
 > Freesurfer already run for OAS30004_MR_d222

In [36]:
sub_batch

['/project_cephfs/3022017.06/OASIS3/subjects/OAS30170_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30612_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30816_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30620_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30752_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30510_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30670_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30715_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30052_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30178_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS31039_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30623_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30089_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30357_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30590_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30650_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30094_MR

## Create Freesurfer longitudinal templates

Note: this needs to be run more carefully than the cross sectional approach above because it just clobbers the template directory. 

In [77]:
sessions

for i in range(len(sessions)):
    if "BL" in sessions[i]:
        bl = sessions[i]

subid + sesid

'000049148426000049148426_FU3'

In [35]:
num_processed = 0
num_failed = 0
num_submitted = 0

verbose = False

log_prefix = 'OASIS3_longsf_'

#sub_batch = sub_dirs[10:]
#sub_batch = sub_dirs_target[5900:]
sub_batch = sub_batch

#fs_env = 'export FREESURFER_HOME=/opt/freesurfer/7.3.2'
fs_env = 'module load freesurfer/7.3.2'

cmd_qsub_fs = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '202400',
                 '-memory', '5gb',
                 '-logfiledir', cluster_log_dir,
                 '-queue verylong'
                ]

print('Processing', len(sub_batch), 'subjects ...')
bad_sessions =[]
for s in sub_batch:
    subid = os.path.splitext(os.path.basename(s))[0] 
    cmd_fs = ['recon-all -base ', subid + '_long',]
    
    sessions = sorted(glob.glob(os.path.join(s+"*")))
    #print(sessions)
    
    # make sure the baseline session is first 
    #for i in range(len(sessions)):
        #if 'MR1' in sessions[i]:
            #bl = sessions[i]
            #del sessions[i]
            #sessions.insert(0, bl)
            #continue
    
    good_sessions = 0 
    for sess in sessions:
        sesid = os.path.basename(sess)
        #full_subid = '_'.join((subid, sesid)) 
        full_subid = sesid
        fs_sess_dir = os.path.join(root_dir, 'freesurfer', full_subid)
        
        fs_sessions = []
        if os.path.exists(os.path.join(fs_sess_dir, 'stats', 'rh.w-g.pct.stats')):
            fs_sessions.append(full_subid)
        else:
            bad_sessions.append(full_subid)
            print(' > Freesurfer did not complete for', full_subid)
            continue
        
        cmd_fs += ['-tp', full_subid]
        good_sessions += 1
        
    if good_sessions < 2:
        print(' > Only one timepoint for', full_subid, 'aborting.')
        continue
        
    cmd_fs += ['-all', '-qcache']
        
    print(' '.join(cmd_fs))

    cmd_env = [fs_env + ' ; ',
               'export SUBJECTS_DIR=' + os.path.join(root_dir,'freesurfer ; ')]
    cmd_str = '"%s"' % str(' '.join(cmd_env + cmd_fs)) 
    cmd_qsub = cmd_qsub_fs + ['-name', log_prefix + str(subid + '_long'),'-command', cmd_str]
    num_submitted += 1
    print(' '.join(cmd_qsub))
    
    subprocess.Popen(' '.join(cmd_qsub), shell=True)

print('jobs submitted =', num_submitted)       
print('number of subjects already processed =',num_processed)
print('number failed =',num_failed)

Processing 1098 subjects ...
 > Only one timepoint for OAS30170_MR_d0005 aborting.
recon-all -base  OAS30612_MR_long -tp OAS30612_MR_d0039 -tp OAS30612_MR_d1201 -all -qcache
/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py -length 202400 -memory 5gb -logfiledir /project_cephfs/3022017.06/OASIS3/logs -queue verylong -name OASIS3_longsf_OAS30612_MR_long -command "module load freesurfer/7.3.2 ;  export SUBJECTS_DIR=/project_cephfs/3022017.06/OASIS3/freesurfer ;  recon-all -base  OAS30612_MR_long -tp OAS30612_MR_d0039 -tp OAS30612_MR_d1201 -all -qcache"
recon-all -base  OAS30816_MR_long -tp OAS30816_MR_d0530 -tp OAS30816_MR_d3379 -all -qcache
/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py -length 202400 -memory 5gb -logfiledir /project_cephfs/3022017.06/OASIS3/logs -queue verylong -name OASIS3_longsf_OAS30816_MR_long -command "module load freesurfer/7.3.2 ;  export SUBJECTS_DIR=/project_cephfs/3022017.06/OASIS3/freesurfer ;  recon-all -base  OAS30816_MR_long -tp

In [25]:
sub_batch

['/project_cephfs/3022017.06/OASIS3/subjects/OAS30170_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30612_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30816_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30620_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30752_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30510_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30670_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30715_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30052_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30178_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS31039_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30623_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30089_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30357_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30590_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30650_MR',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30094_MR

In [32]:
os.path.join(s)

'/project_cephfs/3022017.06/OASIS3/subjects/OAS30119_MR'

## Select subjects

In [12]:
fs_dir = os.path.join(root_dir,'freesurfer')
sub_dirs = glob.glob(os.path.join(fs_dir, '*_long'))

sub_list_target = []
for s in sub_dirs:
    subid = os.path.splitext(os.path.basename(s))[0].split('_')[0] + "_MR"
    
    sub_list_target.append(subid)
        
print(len(sub_list_target), 'subjects found')
sorted(sub_list_target)[:10]
sub_list_target = list(set(sub_list_target))

already_processed = glob.glob(os.path.join(fs_dir, '*.long.*'))
already_processed_list=[]
for s in already_processed:
    processed_subid = os.path.splitext(os.path.basename(s))[0].split('_')[0]
    already_processed_list.append(processed_subid)

len(list(set(already_processed)))
already_processed_list = list(set(already_processed_list))

#len(sub_list_target)
#len(already_processed_list)

leftover_list = [x for x in sub_list_target if x not in already_processed_list]
leftover_list


leftover_list = list(set(leftover_list))

536 subjects found


## Run longitudinal freesurfer

In [16]:
num_processed = 0
num_failed = 0
num_submitted = 0
sub_no_t1 = []

verbose = True

log_prefix = 'OASIS3_longf'

#sub_id_batch = sub_list_target
sub_id_batch = leftover_list

#fs_env = 'export FREESURFER_HOME=/opt/freesurfer/7.3.2'
fs_env = 'module load freesurfer/7.3.2'

cmd_qsub_fs = ['/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py',
                 '-length', '202400',
                 '-memory', '5gb',
                 '-logfiledir', cluster_log_dir,
                 '-queue verylong'
                ]

print('Processing', len(sub_id_batch), 'subjects ...')
for subid in sub_id_batch:
        
    sessions = glob.glob(os.path.join(fs_dir, f'{subid}_*'))
    sessions = [item for item in sessions if 'long' not in item] # remove _long sessions
    
    for full_subid in sessions:
        #sesid = os.path.basename(full_subid)
        
        #if not os.path.exists(os.path.join(fs_dir, full_subid)):
        #    continue
                  
        out_dir = os.path.join(root_dir, 'freesurfer', full_subid, '_longitudinal')
            
        #if os.path.exists(out_dir):
        #    if os.path.exists(os.path.join(out_dir, 'stats', 'rh.w-g.pct.stats')):
        #        # analysis is complete
        #        if verbose:
        #            print(' > Freesurfer already run for', full_subid,'doing nothing')
        #        num_processed += 1
        #        continue
        #    else:
        #         # analysis did not complete
        #        num_failed +=1
        #        if verbose:
        #            print(' > Freesurfer subject',full_subid,'found but failed. Continuing analysis')
        #        cmd_fs = ['recon-all -long ', full_subid, f'{sub_id}_long', '-all','-qcache']
        #else:
            # no analysis has been run
            #if not os.path.exists(t1_file):
            #    if verbose: 
            #        print(' > No T1 found for sub', full_subid, '. Doing nothing')
            #    sub_no_t1.append(subid)
            #    continue
        
        cmd_fs = ['recon-all -long ', full_subid, f'{subid}_long', '-all','-qcache']
            
        #print(' '.join(cmd_fs))

        cmd_env = [fs_env + ' ; ',
                    'export SUBJECTS_DIR=' + fs_dir + ' ; ']
        cmd_str = '"%s"' % str(' '.join(cmd_env + cmd_fs)) 
        cmd_qsub = cmd_qsub_fs + ['-name', log_prefix + str(full_subid),'-command', cmd_str]
    
        num_submitted += 1
        print(' '.join(cmd_qsub))
        
        #subprocess.Popen(' '.join(cmd_qsub), shell=True)

print('jobs submitted =', num_submitted)       
print('number of subjects already processed =',num_processed)
print('number failed =',num_failed)
print('number without t1 =',len(sub_no_t1))

Processing 536 subjects ...
/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py -length 202400 -memory 5gb -logfiledir /project_cephfs/3022017.06/OASIS3/logs -queue verylong -name OASIS3_longf/project_cephfs/3022017.06/OASIS3/freesurfer/OAS30347_MR_d2615 -command "module load freesurfer/7.3.2 ;  export SUBJECTS_DIR=/project_cephfs/3022017.06/OASIS3/freesurfer ;  recon-all -long  /project_cephfs/3022017.06/OASIS3/freesurfer/OAS30347_MR_d2615 OAS30347_MR_long -all -qcache"
/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToCluster.py -length 202400 -memory 5gb -logfiledir /project_cephfs/3022017.06/OASIS3/logs -queue verylong -name OASIS3_longf/project_cephfs/3022017.06/OASIS3/freesurfer/OAS30347_MR_d3101 -command "module load freesurfer/7.3.2 ;  export SUBJECTS_DIR=/project_cephfs/3022017.06/OASIS3/freesurfer ;  recon-all -long  /project_cephfs/3022017.06/OASIS3/freesurfer/OAS30347_MR_d3101 OAS30347_MR_long -all -qcache"
/home/preclineu/andmar/DCCN/Scripts/Torque/SubmitToClust

In [38]:
len(bad_sessions)

['/project_cephfs/3022017.06/OASIS3/subjects/OAS30119_MR_d1209',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30119_MR_d2415',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30119_MR_d3260',
 '/project_cephfs/3022017.06/OASIS3/subjects/OAS30119_MR_d3722']

In [15]:
bad_sessions

['OAS30072_MR_d0218',
 'OAS30156_MR_d0000',
 'OAS30784_MR_d0126',
 'OAS30963_MR_d0047',
 'OAS30144_MR_d1204',
 'OAS31103_MR_d0170',
 'OAS30005_MR_d0143',
 'OAS30092_MR_d0636',
 'OAS30203_MR_d0203',
 'OAS30203_MR_d0802',
 'OAS30259_MR_d0000',
 'OAS30259_MR_d0679',
 'OAS30062_MR_d0087',
 'OAS30745_MR_d2651',
 'OAS30113_MR_d0894',
 'OAS30379_MR_d1169',
 'OAS30408_MR_d1193',
 'OAS30444_MR_d0001',
 'OAS30876_MR_d0126',
 'OAS30941_MR_d0786',
 'OAS30775_MR_d0183',
 'OAS30119_MR_d1209',
 'OAS30960_MR_d0061',
 'OAS30960_MR_d0831',
 'OAS30699_MR_d0108',
 'OAS30022_MR_d1324',
 'OAS30365_MR_d5600',
 'OAS30365_MR_d6199',
 'OAS30012_MR_d0124',
 'OAS30012_MR_d0581',
 'OAS30932_MR_d4007',
 'OAS30935_MR_d0201',
 'OAS30959_MR_d0123',
 'OAS30675_MR_d7433',
 'OAS30516_MR_d0225',
 'OAS30188_MR_d0246',
 'OAS30794_MR_d1680',
 'OAS30581_MR_d0104',
 'OAS30059_MR_d0230',
 'OAS30059_MR_d1188',
 'OAS31081_MR_d0129',
 'OAS30231_MR_d1469',
 'OAS30331_MR_d2229',
 'OAS30852_MR_d2307',
 'OAS30852_MR_d3014',
 'OAS30791